# Keras Functional  and Sequential APIの使い方

## Preparcing Tensorflow2.x and Dataset MNIST

In [1]:
import tensorflow as tf
import keras

# Mnist
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalization 0-255の値が入っているので、0-1に収まるよう正規化します
x_train, x_test = x_train / 255.0, x_test / 255.0

# Check the data
# Now, each row has one data
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


# Sequential の場合の書き方

### ・keras.models.Sequential()にlistで与える

### ・model.add()で1層ずつ足してくかしてモデルをつくる

### 最期に学習条件を決めてcompileすれば完成です。


In [2]:
# Sequentialモデルを定義します

model = keras.models.Sequential()

model.add(tf.keras.layers.Flatten(input_shape =(28, 28)))
model.add(tf.keras.layers.Dense(128, activation ='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation ='softmax'))

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/5
1688/1688 [==============================] - 2s 954us/step - loss: 0.3133 - accuracy: 0.9083 - val_loss: 0.1227 - val_accuracy: 0.9668
Epoch 2/5
1688/1688 [==============================] - 1s 861us/step - loss: 0.1526 - accuracy: 0.9543 - val_loss: 0.0918 - val_accuracy: 0.9740
Epoch 3/5
1688/1688 [==============================] - 1s 852us/step - loss: 0.1172 - accuracy: 0.9649 - val_loss: 0.0790 - val_accuracy: 0.9770
Epoch 4/5
1688/1688 [==============================] - 1s 821us/step - loss: 0.0953 - accuracy: 0.9708 - val_loss: 0.0777 - val_accuracy: 0.9783
Epoch 5/5
313/313 [==============================] - 0s 497us/step - loss: 0.0789 - accuracy: 0.9766
[0.07888970524072647, 0.9765999913215637]


# Functional の場合の書き方

## ①入力が１つの場合

### 上記のSequentialの場合とまったく同じモデルをfunctional APIで書くと次のようになります。

Sequentialだと入力数と出力数がどちらも１つと決まってるのでSequentialでネットワーク構造を定義したら完成でしたが、functional APIだと入力と出力をどちらも複数設定できますので、ネットワーク構造をkeras.layersで定義する部分の２つを書いておいて、入力と出力がいくつあるのかkeras.Model()で定義して完成となります。

In [3]:
# モデル構造を定義します
inputs = tf.keras.layers.Input(shape=(28, 28))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

# 入出力を定義します
model = keras.Model(inputs=inputs, outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3099 - accuracy: 0.9084 - val_loss: 0.1238 - val_accuracy: 0.9662
Epoch 2/5
1688/1688 [==============================] - 1s 805us/step - loss: 0.1490 - accuracy: 0.9558 - val_loss: 0.0909 - val_accuracy: 0.9732
Epoch 3/5
1688/1688 [==============================] - 1s 864us/step - loss: 0.1122 - accuracy: 0.9659 - val_loss: 0.0758 - val_accuracy: 0.9768
Epoch 4/5
1688/1688 [==============================] - 1s 852us/step - loss: 0.0913 - accuracy: 0.9726 - val_loss: 0.0745 - val_accuracy: 0.9795
Epoch 5/5
313/313 [==============================] - 0s 529us/step - loss: 0.0772 - accuracy: 0.9772
[0.0772307813167572, 0.9771999716758728]


(Note)

学習データとテストデータのようにkerasの外からkerasモデルに渡すデータは必ず最初にkeras.layers.Input()で受け取り、そこから加える層の右にその層への入力を（）付きで与えるように書いて、1層ずつ増やしていくという書き方になります。

下の例だとpredictionsに入力から出力までのInput => Flatten => Dense(128, relu) => Dropout => Dense(10, softmax)までのネットワークが全部入ってますので、Sequentialで書いたmodelと同じ内容になります

## ② 入力が2つある場合(出力は１つ)

入力が複数ある場合はinputが複数あるネットワークを書いて、keras.Model()にlistでinputを与えるようにします。下の例はmnistデータを2つに分けてkeras model内で結合してから同じネットワークに通すようにしたものです。

In [4]:
# 複数入力のテストの為にxを分割してみます
# 全ての行、３９２
x_train2_1 = x_train.reshape(60000, 784)[:,:392]  # (60000, 392) 
x_train2_2 = x_train.reshape(60000, 784)[:,392:]  # (60000, 392)

x_test2_1 = x_test.reshape(10000, 784)[:,:392] # (10000, 392)
x_test2_2 = x_test.reshape(10000, 784)[:,392:] # (10000, 392)

# Functional APIでモデルを定義します
input1 = tf.keras.layers.Input(shape=(392,))
input2 = tf.keras.layers.Input(shape=(392,))
inputs = tf.keras.layers.concatenate([input1, input2])

x = tf.keras.layers.Dense(128, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# 入出力を定義します
model = tf.keras.Model(inputs=[input1, input2], outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], y_test))

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 784)          0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 128)          100480      concatenate[0][0]     

None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3099 - accuracy: 0.9092 - val_loss: 0.1282 - val_accuracy: 0.9645
Epoch 2/5
1688/1688 [==============================] - 2s 947us/step - loss: 0.1481 - accuracy: 0.9563 - val_loss: 0.0977 - val_accuracy: 0.9710
Epoch 3/5
1688/1688 [==============================] - 2s 936us/step - loss: 0.1108 - accuracy: 0.9660 - val_loss: 0.0822 - val_accuracy: 0.9767
Epoch 4/5
1688/1688 [==============================] - 2s 895us/step - loss: 0.0902 - accuracy: 0.9723 - val_loss: 0.0801 - val_accuracy: 0.9763
Epoch 5/5
313/313 [==============================] - 0s 589us/step - loss: 0.0720 - accuracy: 0.9780
[0.07204233855009079, 0.9779999852180481]


## ③ 入力と出力が2つある場合（損失関数は１つ）

分岐を加えて出力が2つあるmodelに変えてみました。x1とx2の2つの経路に分岐していて、prediction1とprediction2がそれぞれの出力までのネットワーク情報をもっています。出力段が2つになったのでkeras.Model()に与える出力段も２つになります。

In [5]:
# 複数入力のテストの為にxを分割してみます
x_train2_1 = x_train.reshape(60000, 784)[:,:392]
x_train2_2 = x_train.reshape(60000, 784)[:,392:]
x_test2_1 = x_test.reshape(10000, 784)[:,:392]
x_test2_2 = x_test.reshape(10000, 784)[:,392:]

# Functional APIでモデルを定義します
input1 = keras.layers.Input(shape=(392,))
input2 = keras.layers.Input(shape=(392,))

# Prediction 1
inputs1 = keras.layers.concatenate([input1, input2])
x1 = keras.layers.Dense(128, activation='relu')(inputs1)
x1 = keras.layers.Dropout(0.2)(x1)
prediction1 = keras.layers.Dense(10, activation='softmax')(x1)

# Prediction 2
inputs2 = keras.layers.concatenate([input1, input2])
x2 = keras.layers.Dense(128, activation='relu')(inputs2)
x2 = keras.layers.Dropout(0.2)(x2)
prediction2 = keras.layers.Dense(10, activation='softmax')(x2)

# 入出力を定義します
model = keras.Model(inputs=[input1, input2], outputs=[prediction1, prediction2])

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], [y_train, y_train], 
                 validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], [y_test, y_test]))

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 784)          0           input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 784)          0           input_4[0][0]         

None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.6220 - dense_7_loss: 0.3105 - dense_9_loss: 0.3115 - dense_7_accuracy: 0.9103 - dense_9_accuracy: 0.9091 - val_loss: 0.2464 - val_dense_7_loss: 0.1255 - val_dense_9_loss: 0.1209 - val_dense_7_accuracy: 0.9663 - val_dense_9_accuracy: 0.9685
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2973 - dense_7_loss: 0.1472 - dense_9_loss: 0.1501 - dense_7_accuracy: 0.9560 - dense_9_accuracy: 0.9553 - val_loss: 0.1811 - val_dense_7_loss: 0.0903 - val_dense_9_loss: 0.0908 - val_dense_7_accuracy: 0.9743 - val_dense_9_accuracy: 0.9750
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2228 - dense_7_loss: 0.1111 - dense_9_loss: 0.1117 - dense_7_accuracy: 0.9663 - dense_9_accuracy: 0.9663 - val_loss: 0.1611 - val_dense_7_loss: 0.0830 - val_dense_9_loss: 0.0781 - val_dense_7_accuracy: 0.9752 - val_dense_9_accuracy: 0.9783
Epoch 4/5
1688/1688 [============================

# ④ 入力、出力、損失関数が2つある場合

せっかく出力を分けたので損失関数も別々に入れてみます。

modelを作るときにname=''で名付けておいて、compile()するときにlossを辞書型で渡せば出力ごとに異なる損失関数を使うことができます。下の例だと同じ損失関数を使ってますが、ぜんぜん違う損失関数を指定しても構いません。

学習はトータルの損失関数を最小化するように進めますがデフォルトでは単純に合計するようです。加算比率をloss_weightsに辞書型で渡すことで指定することもできるので、以下では0.5ずつで加算するようにしています。

In [6]:
# 複数入力のテストの為にxを分割してみます
x_train2_1 = x_train.reshape(60000, 784)[:,:392]
x_train2_2 = x_train.reshape(60000, 784)[:,392:]
x_test2_1 = x_test.reshape(10000, 784)[:,:392]
x_test2_2 = x_test.reshape(10000, 784)[:,392:]

# Functional APIでモデルを定義します
input1 = keras.layers.Input(shape=(392,))
input2 = keras.layers.Input(shape=(392,))

# Prediction 1
inputs1 = keras.layers.concatenate([input1, input2])
x1 = keras.layers.Dense(128, activation='relu')(inputs1)
x1 = keras.layers.Dropout(0.2)(x1)
prediction1 = keras.layers.Dense(10, activation='softmax', name='prediction1')(x1)

# Prediction 2
inputs2 = keras.layers.concatenate([input1, input2])
x2 = keras.layers.Dense(128, activation='relu')(inputs2)
x2 = keras.layers.Dropout(0.2)(x2)
prediction2 = keras.layers.Dense(10, activation='softmax', name='prediction2')(x2)

# 入出力を定義します
model = keras.Model(inputs=[input1, input2], outputs=[prediction1, prediction2])


# モデルをcompileします
model.compile(optimizer='adam',
              loss={'prediction1': 'sparse_categorical_crossentropy', 
                    'prediction2': 'sparse_categorical_crossentropy'},
              loss_weights={'prediction1': 0.5,
                            'prediction2': 0.5},
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], [y_train, y_train], 
                 validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], [y_test, y_test]))

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 784)          0           input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 784)          0           input_6[0][0]         

None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3168 - prediction1_loss: 0.3150 - prediction2_loss: 0.3186 - prediction1_accuracy: 0.9086 - prediction2_accuracy: 0.9083 - val_loss: 0.1183 - val_prediction1_loss: 0.1189 - val_prediction2_loss: 0.1177 - val_prediction1_accuracy: 0.9698 - val_prediction2_accuracy: 0.9663
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1531 - prediction1_loss: 0.1515 - prediction2_loss: 0.1547 - prediction1_accuracy: 0.9546 - prediction2_accuracy: 0.9547 - val_loss: 0.0952 - val_prediction1_loss: 0.0962 - val_prediction2_loss: 0.0942 - val_prediction1_accuracy: 0.9705 - val_prediction2_accuracy: 0.9717
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1156 - prediction1_loss: 0.1149 - prediction2_loss: 0.1162 - prediction1_accuracy: 0.9660 - prediction2_accuracy: 0.9649 - val_loss: 0.0798 - val_prediction1_loss: 0.0817 - val_prediction2_loss: 0.0779 - val_prediction1_accur

## ⑤ 学習済みmodelを組み込む

学習済みmodelを部品として組み込むことも出来ます。

使い方はkeras.layersの代わりに学習済みmodelを置くだけですし、組み込んだら1つのkeras modelとして使えますのでアンサンブルモデルも簡潔に書けて便利です。
下の例では上半分で学習しで作ったmodelを下半分で部品として組み込んだmodel2を作っています。


In [7]:
# Functional APIでモデルを定義します
inputs = keras.layers.Input(shape=(28, 28))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# 入出力を定義します
model = keras.Model(inputs=inputs, outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/5
1688/1688 [==============================] - 2s 892us/step - loss: 0.3103 - accuracy: 0.9107 - val_loss: 0.1289 - val_accuracy: 0.9653
Epoch 2/5
1688/1688 [==============================] - 1s 866us/step - loss: 0.1490 - accuracy: 0.9557 - val_loss: 0.0941 - val_accuracy: 0.9743
Epoch 3/5
1688/1688 [==============================] - 1s 854us/step - loss: 0.1121 - accuracy: 0.9661 - val_loss: 0.0822 - val_accuracy: 0.9773
Epoch 4/5
1688/1688 [==============================] - 1s 853us/step - loss: 0.0934 - accuracy: 0.9709 - val_loss: 0.0723 - val_accuracy: 0.9790
Epoch 5/5
313/313 [==============================] - 0s 627us/step - loss: 0.0722 - accuracy: 0.9778
[0.07221358269453049, 0.9778000116348267]


######################################################

In [8]:
# モデルを再利用するモデルを定義します
inputs = keras.layers.Input(shape=(28, 28))
predictions = model(inputs)

# モデルをcompileします
model2 = keras.Model(inputs=inputs, outputs=predictions)
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model2.summary())

# テストデータの予測精度を計算します
print(model2.evaluate(x_test, y_test))

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
functional_9 (Functional)    (None, 10)                101770    
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

313/313 [==============================] - 0s 569us/step - loss: 0.0722 - accuracy: 0.9778
[0.07221358269453049, 0.9778000116348267]


## Appendix: Slice

In [9]:
import numpy as np

data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90,])
data2=np.array(data).reshape(3,3) #3×3の配列を作成
data2

array([[10, 20, 30],
       [40, 50, 60],
       [70, 80, 90]])

In [10]:
data2[:2,] #0～1行目、すべての列

array([[10, 20, 30],
       [40, 50, 60]])

In [11]:
data2[:,:1] #　全ての行、1列目以降

array([[10],
       [40],
       [70]])

In [12]:
data2[:,1:] #すべての行、1列目以降

array([[20, 30],
       [50, 60],
       [80, 90]])

In [13]:
data2[1:,1:] #1行目以降、1列目以降

array([[50, 60],
       [80, 90]])